# Algorithm

#Website Title

In [1]:
## st.title

#Setup LLMs

In [3]:
## Dependencies
!pip install --upgrade --quiet langchain langchain-community langchain-groq neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00


In [1]:
groq_api_key = "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"

In [4]:
!pip install langchain_groq

In [9]:
from langchain_groq import ChatGroq
llm = ChatGroq(groq_api_key=groq_api_key, model_name="llama-3.1-70b-versatile")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7f81cd458670>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f81cd438b20>, model_name='llama-3.1-70b-versatile', groq_api_key=SecretStr('**********'))

#Connect to Neo4j AuraDB

In [10]:
NEO4J_URI = "neo4j+s://82593fd3.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "igPFpSEjY82ZDUDAFc2joQ2TF-_NpzF4DWopg7EmhAw"
NEO4J_DATABASE = "neo4j"

In [11]:
from langchain_community.graphs import Neo4jGraph
kg = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database=NEO4J_DATABASE,
)

#Upload Doc

In [12]:
## Dependencies
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.6 MB/s eta 0:00:00


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
## st.file_uploader
import json
import os
from PyPDF2 import PdfReader

file_name = "/content/drive/MyDrive/Projects/AI/Graph RAG/Resources/Elon_Musk.pdf"
file_as_object = open(file_name, 'rb')
pdf_file = PdfReader(file_as_object)

file_content = ""
for page in pdf_file.pages:
  file_content += page.extract_text()

file_as_object.close()

In [16]:
print(file_content)

Elon Musk
FRS
Musk in 2023
Born Elon Reeve Musk
June 28, 1971
Pretoria, Transvaal, South Africa
CitizenshipSouth Africa
Canada
United States (from 2002)
EducationUniversity of Pennsylvania (BA,
BS)
Title Founder, CEO, and chief engineer
of SpaceX
CEO and product architect of
Tesla, Inc.
Owner, CTO and Executive
Chairman of X (formerly Twitter)
President of the Musk Foundation
Founder of The Boring Company,
X Corp., and xAI
Co-founder of Neuralink, OpenAI,
Zip2, and X.com (part of PayPal)
Elon Musk
Elon Reeve Musk FRS (/ˈiːlɒn/; born June 28, 1971)
is a businessman and investor know n for his key roles
in space company SpaceX and automotive company
Tesla, Inc. Other involvements include ownership of
X Corp., the company that operates the social media
platform X (formerly know n as Twitter), and his role
in the foundi ng of The Boring Company, xAI,
Neuralink and OpenAI. He is one of the wealthiest
people in the world; as of Augus t 2024 , Forbes
estimates his net worth to be US$241 bi ll

#Chunking and Vector Indexing

In [19]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.5 MB/s eta 0:00:00


In [22]:
## Convert the file_content into chunks and nodes
from langchain.text_splitter import TokenTextSplitter
if file_content is not None:
  text_splitter = TokenTextSplitter(
      chunk_size = 512,
      chunk_overlap  = 50,
  )

  text_chunks = text_splitter.split_text(file_content)
  print(len(text_chunks))

226


#Delete the database in AuraDB

In [ ]:
## If Uploaded_Doc is not None then delete the existing data in AuraDB

In [ ]:
## Create class or function to connect to AuraDB

In [ ]:
## Create a function with delete db cypher query being executed on the Initializies instance and return count_nodes (=0) and define it as st.session_state

#Convert the Doc to Knowledge Graph and then Display Graph

In [2]:
# When the count_nodes is returned enable a Show Graph button

In [3]:
# Upon click of the Show Graph button, a function Show_graph is called. As a result a loading spinner is displayed

In [4]:
## 1. The uploaded docs is converted to chunks and nodes

In [6]:
## 2. The docs are converted using llm_graph_transformer by langchain and stored as python object

In [7]:
## 3. The Neo4j database is populated with the llm generated KG

In [ ]:
## 4. A widget is returned for calling the Show_graph()

In [5]:
## 5. The KG is vectorized

# Chat with KG

In [8]:
## After displaying the graph widget, give a button for Ask your KG

In [9]:
## Setup a chat prompt template for system prompt and user prompt

In [10]:
## Convert the user queries to cypher

In [11]:
## Create a chain using KG|cypher_prompt|LLM

In [12]:
## chain.invoke for querying from KG